#Complete_code_jobcv using BERT and skills extraction by pyresparser.

#Dropped becuase processing was taking 4 sec for each resume



In [ ]:
!pip install -q PyPDF2 sentence-transformers pandas openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.2 MB/s eta 0:00:00


In [ ]:
import os
import re
import pandas as pd
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
import os
import nltk

# Install SpaCy Dependencies
os.system('python -m pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz')

# Install nltk Dependencies
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    """
    Preprocess text by removing special characters, lowercasing, and trimming.
    """
    # Convert to lowercase
    text = text.lower()
    # Remove special characters, digits, and extra spaces
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    """
    Extract text content from a PDF file using PyPDF2.
    """
    try:
        reader = PdfReader(pdf_path)
        text = " ".join(page.extract_text() for page in reader.pages if page.extract_text())
        return text
    except Exception as e:
        print(f"Error reading PDF {pdf_path}: {e}")
        return None

# Function to extract relevant sections from resumes
def extract_relevant_resume_sections(text):
    """
    Extract relevant sections (skills, experience, education, and projects) from a resume.
    Targets a variety of section headers.
    """
    sections = {"skills": "", "experience": "", "education": "", "projects": ""}

    # Skills Section
    skills_match = re.search(r'(skills|technical skills|core competencies|areas of expertise)(.*?)(experience|education|projects|summary|$)', text, re.DOTALL)
    if skills_match:
        sections["skills"] = skills_match.group(2).strip()

    # Experience Section
    experience_match = re.search(r'(experience|work history|employment history|professional experience|career summary)(.*?)(education|skills|projects|summary|$)', text, re.DOTALL)
    if experience_match:
        sections["experience"] = experience_match.group(2).strip()

    # Education Section
    education_match = re.search(r'(education|academic background|qualifications|academic qualifications)(.*?)(experience|skills|projects|summary|$)', text, re.DOTALL)
    if education_match:
        sections["education"] = education_match.group(2).strip()

    # Projects Section (Optional)
    projects_match = re.search(r'(projects|portfolio|notable work|key projects|case studies)(.*?)(experience|skills|education|summary|$)', text, re.DOTALL)
    if projects_match:
        sections["projects"] = projects_match.group(2).strip()

    # Combine all relevant sections
    combined_text = " ".join([sections["skills"], sections["experience"], sections["education"], sections["projects"]])
    return preprocess_text(combined_text)

# Function to extract relevant sections from a job description
def extract_relevant_job_description_sections(text):
    """
    Extract relevant sections (responsibilities, requirements, qualifications, skills) from a job description.
    Targets a variety of section headers.
    """
    # Responsibilities Section
    responsibilities_match = re.search(r'(responsibilities|duties|role|tasks|job duties|job responsibilities)(.*?)(requirements|qualifications|skills|summary|desired skills|$)', text, re.DOTALL)
    responsibilities = responsibilities_match.group(2).strip() if responsibilities_match else ""

    # Requirements or Qualifications Section
    requirements_match = re.search(r'(requirements|qualifications|skills|desired skills|eligibility criteria|job requirements)(.*?)(responsibilities|duties|summary|$)', text, re.DOTALL)
    requirements = requirements_match.group(2).strip() if requirements_match else ""

    # Skills Section (Optional)
    skills_match = re.search(r'(skills|technical skills|core competencies|key qualifications|knowledge and abilities|preferred skills)(.*?)(responsibilities|requirements|summary|$)', text, re.DOTALL)
    skills = skills_match.group(2).strip() if skills_match else ""

    # Combine extracted sections
    combined_text = " ".join([responsibilities, requirements, skills])
    return preprocess_text(combined_text)

# Function to load and preprocess resumes
def load_resumes(folder_path):
    """
    Load all resumes from the specified folder, extract relevant sections, preprocess, and return as a list of tuples.
    Each tuple contains (filename, preprocessed_text).
    """
    resumes = []
    for file in os.listdir(folder_path):
        if file.endswith(".pdf"):
            file_path = os.path.join(folder_path, file)
            raw_text = extract_text_from_pdf(file_path)
            if raw_text:
                relevant_text = extract_relevant_resume_sections(raw_text)
                resumes.append((file, relevant_text))
                ################################extra
                print("Resume text")
                print(relevant_text)
    print(resumes)
    return resumes

# Function to load and preprocess job description
def load_job_description(file_path):
    """
    Load a job description from a text file, extract relevant sections, preprocess, and return the text.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            raw_text = f.read()
            relevant_text = extract_relevant_job_description_sections(raw_text)
            print("Job decription text")
            ################################extra
            print(relevant_text)
            return relevant_text
    except Exception as e:
        print(f"Error reading job description {file_path}: {e}")
        return None

# Main processing function
def match_resumes_to_job_description(resume_folder, job_description_file, output_excel):
    """
    Match resumes to the job description, calculate similarity scores, and save results to Excel.
    """
    # Load the model
    model = SentenceTransformer('bwbayu/sbert_model_jobcv') #pretrained bert for cv
    # model = SentenceTransformer('all-MiniLM-L6-v2') #good
    # model = SentenceTransformer('paraphrase-MiniLM-L6-v2') bad
    #model = SentenceTransformer('all-MiniLM-L12-v2')



    # Load and preprocess resumes
    resumes = load_resumes(resume_folder)
    if not resumes:
        print("No valid resumes found in the folder.")
        return

    # Load and preprocess job description
    job_description = load_job_description(job_description_file)
    if not job_description:
        print("Job description file could not be processed.")
        return

    # Encode the job description
    job_embedding = model.encode([job_description])[0]

    # Process each resume and calculate similarity
    results = []
    for filename, resume_text in resumes:
        resume_embedding = model.encode([resume_text])[0]
        similarity_score = cosine_similarity([job_embedding], [resume_embedding])[0][0]
        results.append((filename, similarity_score))

    # Sort results by similarity score in descending order
    results = sorted(results, key=lambda x: x[1], reverse=True)

    # Save results to an Excel file
    results_df = pd.DataFrame(results, columns=["Resume Filename", "Similarity Score"])
    results_df.to_excel(output_excel, index=False, engine='openpyxl')
    print(f"Results saved to {output_excel}")




In [ ]:
# Example usage in Google Colab
if __name__ == "__main__":
    # Define paths
    resume_folder = "/content/resumes"  # Path to the folder containing PDF resumes
    job_description_file = "/content/JD.txt"  # Path to the job description file
    #output_excel = "/content/similarity_results.xlsx"  # Path to save the results
    output_excel = "/content/test.xlsx" #test #########################extra

    # Ensure directories/files exist
    if not os.path.exists(resume_folder):
        print(f"Resume folder not found: {resume_folder}")
    elif not os.path.exists(job_description_file):
        print(f"Job description file not found: {job_description_file}")
    else:
        # Run the matching process
        match_resumes_to_job_description(resume_folder, job_description_file, output_excel)

Resume text
work experience graphic designer brightcreations agency los angeles ca designed visual assets for social media campaigns and client presentations collaborated with the marketing team to develop branding strategies for clients marketing intern brandfocus san diego ca assisted in running social media campaigns on facebook and instagram created promotional materials using canva and indesign education bachelor s in graphic design california college of the arts com objective creative professional with a passion for design looking for opportunities in graphic design or branding skills adobe photoshop illustrator and after effects basic knowledge of wordpress and seo principles excellent eye for detail and strong communication
Resume text
looking to grow in the field of analytics and data science skills python pandas matplotlib sql some r basic machine learning logistic regression decision trees familiar with tableau excel pivottables work experience data analyst dataworks inc chi

# END


# Resume Parser

In [ ]:
!pip install pyresparser

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 9.7 MB/s eta 0:00:00
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3960 sha256=f5ce1f50eebe36bb0d5c8b0b7b5b2f2cb7fbb27d55f49d9d5ebd16065555db82
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt


In [ ]:
import warnings

# Suppress specific warnings from SpaCy
warnings.filterwarnings("ignore", message=r"\[W031\] Model '.*' requires spaCy v.*", category=UserWarning)


import os
import re
import pandas as pd
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from pyresparser import ResumeParser
from multiprocessing import cpu_count, Pool
from pyresparser.utils import extract_skills
import spacy


# Function to preprocess text
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    try:
        reader = PdfReader(pdf_path)
        text = " ".join(page.extract_text() for page in reader.pages if page.extract_text())
        return text
    except Exception as e:
        print(f"Error reading PDF {pdf_path}: {e}")
        return None


# Function to extract relevant sections from resumes
def extract_relevant_resume_sections(text):
    sections = {"skills": "", "experience": "", "education": "", "projects": ""}
    skills_match = re.search(r'(skills|technical skills|core competencies|areas of expertise)(.*?)(experience|education|projects|summary|$)', text, re.DOTALL)
    if skills_match:
        sections["skills"] = skills_match.group(2).strip()
    experience_match = re.search(r'(experience|work history|employment history|professional experience|career summary)(.*?)(education|skills|projects|summary|$)', text, re.DOTALL)
    if experience_match:
        sections["experience"] = experience_match.group(2).strip()
    education_match = re.search(r'(education|academic background|qualifications|academic qualifications)(.*?)(experience|skills|projects|summary|$)', text, re.DOTALL)
    if education_match:
        sections["education"] = education_match.group(2).strip()
    projects_match = re.search(r'(projects|portfolio|notable work|key projects|case studies)(.*?)(experience|skills|education|summary|$)', text, re.DOTALL)
    if projects_match:
        sections["projects"] = projects_match.group(2).strip()
    combined_text = " ".join([sections["skills"], sections["experience"], sections["education"], sections["projects"]])
    return preprocess_text(combined_text)


# Function to extract relevant sections from a job description
def extract_relevant_job_description_sections(text):
    responsibilities_match = re.search(r'(responsibilities|duties|role|tasks|job duties|job responsibilities)(.*?)(requirements|qualifications|skills|summary|desired skills|$)', text, re.DOTALL)
    responsibilities = responsibilities_match.group(2).strip() if responsibilities_match else ""
    requirements_match = re.search(r'(requirements|qualifications|skills|desired skills|eligibility criteria|job requirements)(.*?)(responsibilities|duties|summary|$)', text, re.DOTALL)
    requirements = requirements_match.group(2).strip() if requirements_match else ""
    skills_match = re.search(r'(skills|technical skills|core competencies|key qualifications|knowledge and abilities)(.*?)(responsibilities|requirements|summary|$)', text, re.DOTALL)
    skills = skills_match.group(2).strip() if skills_match else ""
    combined_text = " ".join([responsibilities, requirements, skills])
    return preprocess_text(combined_text)


# Function to load and preprocess resumes
def load_resumes(folder_path):
    resumes = []
    for file in os.listdir(folder_path):
        if file.endswith(".pdf"):
            file_path = os.path.join(folder_path, file)
            raw_text = extract_text_from_pdf(file_path)
            if raw_text:
                relevant_text = extract_relevant_resume_sections(raw_text)
                resumes.append((file, relevant_text))
    return resumes


# Function to load and preprocess job description
def load_job_description(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            raw_text = f.read()
            relevant_text = extract_relevant_job_description_sections(raw_text)
            return relevant_text
    except Exception as e:
        print(f"Error reading job description {file_path}: {e}")
        return None


# Skill score calculation functions
def get_candidate_score(job_skill_count, job_skills, candidate_skills):
    common_skills = job_skills.intersection(candidate_skills)
    candidate_score = float(len(common_skills)) / job_skill_count
    return candidate_score * 100


def get_candidate_score_wrapper(args):
    return get_candidate_score(*args)


def sort_candidates(job_desc_text, candidates_df):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(job_desc_text)
    job_skills = set([skill.lower() for skill in extract_skills(doc, doc.noun_chunks)])
    job_skill_count = len(job_skills)
    candidates_skills = [
        set([skill.strip().lower() for skill in skill_list.split(",")])
        for skill_list in candidates_df["Skills"].values.tolist()
    ]
    processing_data = [(job_skill_count, job_skills, person_skills) for person_skills in candidates_skills]
    with Pool(cpu_count()) as process_pool:
        candidates_df["Score"] = process_pool.map(get_candidate_score_wrapper, processing_data)
    return candidates_df


# Main function to process resumes and combine results
def process_resumes(resume_folder, job_description_file, output_csv, output_excel):
    # Load the model
    bert_model = SentenceTransformer('all-MiniLM-L6-v2')

    # Load and preprocess job description
    job_description = load_job_description(job_description_file)
    if not job_description:
        print("Job description file could not be processed.")
        return

    # Load and preprocess resumes
    resumes = load_resumes(resume_folder)
    if not resumes:
        print("No valid resumes found in the folder.")
        return

    # Encode the job description
    job_embedding = bert_model.encode([job_description])[0]

    # Process each resume and calculate scores
    results = []
    extracted_data = []
    for filename, resume_text in resumes:
        resume_embedding = bert_model.encode([resume_text])[0]
        similarity_score = cosine_similarity([job_embedding], [resume_embedding])[0][0]

        # Extract skills using pyresparser with error handling
        try:
            resume_data = ResumeParser(os.path.join(resume_folder, filename)).get_extracted_data()
            candidate_skills = ", ".join(resume_data.get("skills", []))
            extracted_data.append({
                "Filename": filename,
                **resume_data
            })
        except Exception as e:
            print(f"Error parsing {filename}: {e}")
            candidate_skills = ""

        candidates_df = pd.DataFrame({"Skills": [candidate_skills]})
        ranked_df = sort_candidates(job_description, candidates_df)

        # Get skill score and normalize
        skill_score = ranked_df.iloc[0]["Score"] / 100

        # Calculate final score
        final_score = 0.7 * similarity_score + 0.3 * skill_score
        results.append({
            "Resume Filename": filename,
            "BERT Similarity": similarity_score,
            "Skill Score": skill_score,
            "Final Score": final_score
        })

    # Save results to CSV
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values(by="Final Score", ascending=False)  # Sort by Final Score in descending order
    results_df.to_csv(output_csv, index=False)
    print(f"Results saved to {output_csv}")

    # Save extracted data to Excel
    extracted_data_df = pd.DataFrame(extracted_data)
    extracted_data_df.to_excel(output_excel, index=False)
    print(f"Extracted data saved to {output_excel}")


# Example usage
if __name__ == '__main__':
    process_resumes("/content/resumes", "/content/job_description.txt", "final_results.csv", "extracted_data.xlsx")


/usr/local/lib/python3.10/dist-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.9). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/usr/local/lib/python3.10/dist-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.9). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/usr/local/lib/python3.10/dist-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_training' (

Error parsing AshadAbdullah Resume - K213296 Ashad Abdullah Qureshi.pdf: 'PDFObjRef' object is not iterable


/usr/local/lib/python3.10/dist-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.9). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/usr/local/lib/python3.10/dist-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.9). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/usr/local/lib/python3.10/dist-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_training' (

Results saved to final_results.csv
Extracted data saved to extracted_data.xlsx
